In [3]:
import uproot
import numpy as np
import awkward
import concurrent.futures
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [4]:
executor = concurrent.futures.ThreadPoolExecutor(8)
filename = '4gev_1e_tskim_v12_kaons_ldmx-det-v12_toughkskim.root'

# radius of containment for each ECal layer
radius_beam_68 = [4.73798004, 4.80501156, 4.77108164, 4.53839401, 4.73273021,
4.76662872, 5.76994967, 5.92028271, 7.28770932, 7.60723209,
9.36050277, 10.03247442, 12.14656399, 13.16076587, 15.88429816,
17.03559932, 20.32607264, 21.75096888, 24.98745754, 27.02031225,
30.78043038, 33.03033267, 37.55088662, 40.14062264, 47.95964745,
55.96441035, 66.33128366, 70.42649416, 86.68563278, 102.49022815,
119.06854141, 121.20048803, 127.5236134, 121.99024095]

# distance between each ECal layer
layer_dz = np.array([7.850, 13.300, 26.400, 33.500, 47.950, 56.550, 72.250, 81.350, 97.050, 106.150,
            121.850, 130.950, 146.650, 155.750, 171.450, 180.550, 196.250, 205.350, 221.050,
            230.150, 245.850, 254.950, 270.650, 279.750, 298.950, 311.550, 330.750, 343.350,
            362.550, 375.150, 394.350, 406.950, 426.150, 438.750])

# z-position of each ECal layer
layer_z = 240.5 + layer_dz

# list of branches we want to store from the root file
branches = []
branch_suffix = 'v3_v12'

# branchnames
ecal_branch = 'EcalRecHits_{}/EcalRecHits_{}'.format(branch_suffix, branch_suffix)
ecalSP_branch = 'EcalScoringPlaneHits_{}/EcalScoringPlaneHits_{}'.format(branch_suffix, branch_suffix)
tSP_branch = 'TargetScoringPlaneHits_{}/TargetScoringPlaneHits_{}'.format(branch_suffix, branch_suffix)
ecalVeto_branch = 'EcalVeto_{}'.format(branch_suffix)
ecalSim_branch = 'EcalSimHits_{}/EcalSimHits_{}'.format(branch_suffix, branch_suffix)

In [5]:
# add EcalRecHits branches
for leaf in ['xpos_', 'ypos_', 'zpos_', 'energy_']:
    branches.append('{}.{}'.format(ecal_branch, leaf))

# add EcalVeto branches
for leaf in ['showerRMS_', 'epAng_', 'passesVeto_']:
    branches.append('{}/{}'.format(ecalVeto_branch, leaf))

# add EcalSPHits and TargetSPHits branches
# Use EcalSPHits for fiducial/non-fiducial and TargetSPHits for trajectories
for leaf in ['x_', 'y_', 'z_', 'px_', 'py_', 'pz_', 'pdgID_', 'trackID_']:
    branches.append('{}.{}'.format(ecalSP_branch, leaf))
    branches.append('{}.{}'.format(tSP_branch, leaf))

# add the eventNumber leaf
branches.append('EventHeader/eventNumber_')

In [6]:
# use uproot and load all of the data into a dictionary: {'leaf1': [data], 'leaf2': [data], ...}
t = uproot.open(filename)['LDMX_Events']
table = t.arrays(expressions=branches, interpretation_executor=executor)

# we store all of our data into the dictionary "tree"
tree = {}
for branch in branches:
    tree[branch] = table[branch]

#print(tree)


In [7]:
# find the number of events that pass through the vetoes
# record the event numbers of the events that pass through the vetoes
events = 0
eventNumbers = []
for event in range(len(tree['EventHeader/eventNumber_'])):
    events +=1
    eventNumbers.append(tree['EventHeader/eventNumber_'][event])

#print(events)
#print(eventNumbers)


In [33]:
# calculate the electron and photon trajectories

# we will store the data for the trajectory positions inside this dictionary:
trajectories = {}
b1 = []
b2 = []
b3 = []
b4 = []

# loop through all of the events
for i in range(len(tree['EventHeader/eventNumber_'])):
    tSPHits = {}
    ecalSPHits = {}
    for x in ['x_', 'y_', 'z_', 'px_', 'py_', 'pz_', 'pdgID_', 'trackID_']:
        tSPHits[x] = (table['{}.{}'.format(tSP_branch, x)])[i]
        ecalSPHits[x] = (table['{}.{}'.format(ecalSP_branch, x)])[i]
   
    # find the max pz at the target scoring plane for the recoil electron
    max_pz = 0
    r = 0
    for j in range(len(tSPHits['z_'])):
        if tSPHits['pdgID_'][j] == 11 and tSPHits['z_'][j] > 4.4 and tSPHits['z_'][j] < 4.6 and tSPHits['pz_'][j] > max_pz and tSPHits['trackID_'][j] == 1:
            max_pz = tSPHits['pz_'][j]
            r = j
    #print("Found recoil e- pz at target for event {}: {}".format(tree['EventHeader/eventNumber_'][i],max_pz))

    # find the max pz at the ecal face for the recoil electron
    max_pz_e = 0
    r_e = 0
    for j in range(len(ecalSPHits['z_'])):
        if ecalSPHits['pdgID_'][j] == 11 and ecalSPHits['z_'][j] > 239 and ecalSPHits['z_'][j] < 242 and ecalSPHits['pz_'][j] > max_pz_e and ecalSPHits['trackID_'][j] == 1:
            max_pz_e = ecalSPHits['pz_'][j]
            r_e = j
    fiducial = max_pz_e != 0
    
    if fiducial:
        E_beam = 4000
        target_dist = 241.5
        # positions and trajectory vectors of recoil e- at Ecal SP
        etraj_sp = np.array((ecalSPHits['x_'][r_e], ecalSPHits['y_'][r_e], ecalSPHits['z_'][r_e]))
        enorm_sp = np.array((ecalSPHits['px_'][r_e]/ecalSPHits['pz_'][r_e], ecalSPHits['py_'][r_e]/ecalSPHits['pz_'][r_e], 1.0))
        # positions and trajectory vectors of photon at Target SP
        pnorm_sp = np.array((-tSPHits['px_'][r]/(E_beam - tSPHits['pz_'][r]), -tSPHits['py_'][r]/(E_beam - tSPHits['pz_'][r]), 1.0))
        ptraj_sp = np.array((tSPHits['x_'][r] + target_dist*pnorm_sp[0], tSPHits['y_'][r] + target_dist*pnorm_sp[1], tSPHits['z_'][r] + target_dist))
        # store the trajectory information 
        b1.append(etraj_sp)
        b2.append(enorm_sp)
        b3.append(pnorm_sp)
        b4.append(ptraj_sp)
    else:
        etraj_sp, enorm_sp, ptraj_sp, pnorm_sp = None, None, None, None

# store the e- trajectory and its normal vector, store the photon trajectory and its normal vector
trajectories['etraj_sp'] = b1
trajectories['enorm_sp'] = b2
trajectories['pnorm_sp'] = b3
trajectories['ptraj_sp'] = b4

print(len(trajectories['enorm_sp']))

35


In [37]:
# loop through each event and cut all of the hits within electron containment radii

# this is a new dictionary that will story only the hits that are outside the e- containment radii
cutTree = {}

eventsX = []
eventsY = []
eventsZ = []

# loop through each event 
for event in range(len(tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'])):
    ecal_front = 240.5
    etraj_front = np.array(trajectories['etraj_sp']) 
    ptraj_front = np.array(trajectories['ptraj_sp']) 

    # obtain the event's e- trajectory layer intercepts
    eLayerIntercepts = []
    intercept = (trajectories['etraj_sp'][event][0] + layer_dz*trajectories['enorm_sp'][event][0], 
                trajectories['etraj_sp'][event][1] + layer_dz*trajectories['enorm_sp'][event][1], 
                trajectories['etraj_sp'][event][2] + layer_dz*trajectories['enorm_sp'][event][2])
    eLayerIntercepts.append(intercept)
    eLayerIntercepts = np.array(eLayerIntercepts)    

    # obtain the event's photon trajectory layer intercepts
    pLayerIntercepts = []
    intercept = (trajectories['ptraj_sp'][event][0] + layer_dz*trajectories['pnorm_sp'][event][0], 
                trajectories['ptraj_sp'][event][1] + layer_dz*trajectories['pnorm_sp'][event][1], 
                trajectories['ptraj_sp'][event][2] + layer_dz*trajectories['pnorm_sp'][event][2])
    pLayerIntercepts.append(intercept)
    pLayerIntercepts = np.array(pLayerIntercepts)
    
    hitsX = []
    hitsY = []
    hitsZ = []

    # loop through each hit within the event
    for hit in range(len(tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'][event])):
        x = tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'][event][hit]
        y = tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.ypos_'][event][hit]
        z = tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.zpos_'][event][hit]
        
        # loop through each layer of the ECal and check if its distance from the e- layer intercept is > corresponding layer's containment radius
        for layer in range(34):
            if z >= layer_z[layer] - .25 and z <= layer_z[layer] + .25:
                dist = np.sqrt((eLayerIntercepts[0][0][layer] - x)**2 + (eLayerIntercepts[0][1][layer] - y)**2)
                if dist >= radius_beam_68[layer]:
                    #print("({},{}), eLayerIntercept: ({},{})".format(x,y,eLayerIntercepts[0][0][layer],eLayerIntercepts[0][1][layer]))

                    hitsX.append(x)
                    hitsY.append(y)
                    hitsZ.append(z)

    eventsX.append(hitsX)
    eventsY.append(hitsY)
    eventsZ.append(hitsZ)

# these are our new branches without the hits inside the electron containment radius
cutTree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'] = eventsX
cutTree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.ypos_'] = eventsY
cutTree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.zpos_'] = eventsZ

for i in range(len(tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'])):
    Hits1 = len(tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'][i])
    Hits2 = len(cutTree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'][i])
    print('For event {}: Original Hits = {}, Outside Radii Hits = {}'.format(i, Hits1, Hits2))




For event 0: Original Hits = 46, Outside Radii Hits = 31
For event 1: Original Hits = 44, Outside Radii Hits = 31
For event 2: Original Hits = 55, Outside Radii Hits = 41
For event 3: Original Hits = 32, Outside Radii Hits = 29
For event 4: Original Hits = 38, Outside Radii Hits = 21
For event 5: Original Hits = 37, Outside Radii Hits = 23
For event 6: Original Hits = 34, Outside Radii Hits = 22
For event 7: Original Hits = 38, Outside Radii Hits = 29
For event 8: Original Hits = 34, Outside Radii Hits = 23
For event 9: Original Hits = 32, Outside Radii Hits = 22
For event 10: Original Hits = 22, Outside Radii Hits = 17
For event 11: Original Hits = 44, Outside Radii Hits = 34
For event 12: Original Hits = 40, Outside Radii Hits = 26
For event 13: Original Hits = 45, Outside Radii Hits = 29
For event 14: Original Hits = 30, Outside Radii Hits = 20
For event 15: Original Hits = 51, Outside Radii Hits = 30
For event 16: Original Hits = 32, Outside Radii Hits = 26
For event 17: Original H

In [11]:
# Code for writing the hit positions into a .dat file

x1 = list(eLayerIntercepts[0][0])
y1 = list(eLayerIntercepts[0][1])
z1 = list(eLayerIntercepts[0][2])


file = open('test.dat', "w")
for i in range(len(x1)):
    file.write(str(x1[i]) + ", " + str(y1[i]) + ", " + str(z1[i]) + "\n") 
file.close()


In [32]:
# Plot all of the points and trajectories (FOR DEBUGGING PURPOSES)
import plotly.graph_objects as go
import plotly.express as px

# for event 2
x_vals = []
y_vals = []
z_vals = []

# store all of the position values
for hit in range(len(tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'])):
    x_vals.append(tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.xpos_'][1][hit])
    y_vals.append(tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.ypos_'][1][hit])
    z_vals.append(tree['EcalRecHits_v3_v12/EcalRecHits_v3_v12.zpos_'][1][hit])

df = {}
df['x'] = x_vals
df['y'] = y_vals
df['z'] = z_vals


eLayerIntercepts = []
intercept = (trajectories['etraj_sp'][0][0] + layer_dz*trajectories['enorm_sp'][0][0], 
                trajectories['etraj_sp'][0][1] + layer_dz*trajectories['enorm_sp'][0][1], 
                trajectories['etraj_sp'][0][2] + layer_dz*trajectories['enorm_sp'][0][2])
eLayerIntercepts.append(intercept)

traj = {}
traj['trajx'] = eLayerIntercepts[0][0]
traj['trajy'] = eLayerIntercepts[0][1]
traj['trajz'] = eLayerIntercepts[0][2]

# radius of containment
circle = {}
radius = radius_beam_68[0]
centerX = eLayerIntercepts[0][0][0]
centerY = eLayerIntercepts[0][1][0]
centerZ = eLayerIntercepts[0][2][0]
theta = np.linspace(0, 2*np.pi, 360)
circle['circleX'] = radius * np.cos(theta) + centerX
circle['circleY'] = radius * np.sin(theta) + centerY
circle['circleZ'] = np.zeros(360) + centerZ
fig4 = px.line_3d(circle, x='circleX', y='circleY', z='circleZ')

circle2 = {}
radius = radius_beam_68[1]
centerX = eLayerIntercepts[0][0][1]
centerY = eLayerIntercepts[0][1][1]
centerZ = eLayerIntercepts[0][2][1]
theta = np.linspace(0, 2*np.pi, 360)
circle2['circleX'] = radius * np.cos(theta) + centerX
circle2['circleY'] = radius * np.sin(theta) + centerY
circle2['circleZ'] = np.zeros(360) + centerZ
fig5 = px.line_3d(circle2, x='circleX', y='circleY', z='circleZ')

circle3 = {}
radius = radius_beam_68[2]
centerX = eLayerIntercepts[0][0][2]
centerY = eLayerIntercepts[0][1][2]
centerZ = eLayerIntercepts[0][2][2]
theta = np.linspace(0, 2*np.pi, 360)
circle3['circleX'] = radius * np.cos(theta) + centerX
circle3['circleY'] = radius * np.sin(theta) + centerY
circle3['circleZ'] = np.zeros(360) + centerZ
fig6 = px.line_3d(circle3, x='circleX', y='circleY', z='circleZ')

circle4 = {}
radius = radius_beam_68[3]
centerX = eLayerIntercepts[0][0][3]
centerY = eLayerIntercepts[0][1][3]
centerZ = eLayerIntercepts[0][2][3]
theta = np.linspace(0, 2*np.pi, 360)
circle4['circleX'] = radius * np.cos(theta) + centerX
circle4['circleY'] = radius * np.sin(theta) + centerY
circle4['circleZ'] = np.zeros(360) + centerZ
fig7 = px.line_3d(circle4, x='circleX', y='circleY', z='circleZ')

circle5 = {}
radius = radius_beam_68[4]
centerX = eLayerIntercepts[0][0][4]
centerY = eLayerIntercepts[0][1][4]
centerZ = eLayerIntercepts[0][2][4]
theta = np.linspace(0, 2*np.pi, 360)
circle5['circleX'] = radius * np.cos(theta) + centerX
circle5['circleY'] = radius * np.sin(theta) + centerY
circle5['circleZ'] = np.zeros(360) + centerZ
fig8 = px.line_3d(circle5, x='circleX', y='circleY', z='circleZ')

fig1 = px.scatter_3d(df,x='x', y='y', z='z', labels={
                     "x": "X (mm)",
                     "y": "Y (mm)",
                     "z": "Z (mm)"}, width=700,height=700)

fig2 = px.line_3d(traj, x="trajx", y="trajy", z="trajz", labels={
                     "trajx": "X (mm)",
                     "trajy": "Y (mm)",
                     "trajz": "Z (mm)"})

fig3 = go.Figure(data=fig1.data + fig2.data+ fig4.data + fig5.data + fig6.data + fig7.data + fig8.data)
fig3.update_xaxes(range=[-100, 100])
fig3.update_yaxes(range=[-100, 100])
fig3.update_layout(
    scene = dict(
        xaxis = dict(nticks=6, range=[-100,100],),
                     yaxis = dict(nticks=10, range=[-100,100],),
                     zaxis = dict(nticks=10, range=[240,300],),),
    autosize=False,
    margin=dict(r=20, l=10, b=10, t=10))
fig3.update_traces(marker=dict(size=3,
                              line=dict(width=2,
                                        color='Red')),
                  selector=dict(mode='markers'))


fig3.show()